#### Notebook dedicado a realizar testes com uma nova abordagem para o problema de classificação de gêneros musicais: aumentar o dataset dividindo o conjunto em N vezes, com janelas fixas de X segundos e um overlap de Y segundos. 
##### Parâmetros a serem otimizados:
- O tempo da janela (ex.: 3,6,15 segundos)
- O tamanho do overlap: a depender do quanto se deseja aumentar o dataset;


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import pickle
from sklearn.ensemble import GradientBoostingClassifier
import librosa
import os
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from scipy.fft import fft
from scipy.signal import spectrogram
from scipy.stats import kurtosis
from scipy.stats import linregress
from scipy.stats import skew
from scipy.stats import entropy

#### Função que extrai com janelas de N segundos e overlap de M segundos:
##### TESTE 1: N = 3s, M = 0.75s


In [58]:
nperseg=2048
def features_overlap(n_mfcc=25,N=3,M=0.75):
    
    overlap = np.floor(M * 22050); wsize = N * 22050; ## numero de segundos * fs = numero de amostras
    maxN = 30 * 22050;
    n_windows = np.floor((30-N)/M) + 1

    import os
    general_path = '/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original'
    os.chdir(general_path)

    def compute_features(y,sr, i, folder,file):
        
        _, _, spectrogram_data = spectrogram(x=y, fs=sr, nperseg=2048)
        spectrogram_data = np.abs(spectrogram_data)  # Take the absolute value of the spectrogram
        power_spectrum = np.mean(spectrogram_data, axis=1)
        normalized_spectrum = power_spectrum / np.sum(power_spectrum)
        frequency_axis = np.fft.rfftfreq(nperseg, 1 / sr)
        log_power_spectrum = np.log10(power_spectrum)
        energy = np.sum(spectrogram_data, axis=1)
        n_frames = energy.shape[0]
        
        spectral_entropy = entropy(normalized_spectrum, base=2)
        spread = np.sqrt(np.sum((frequency_axis[:, np.newaxis] - np.mean(frequency_axis)) ** 2 * power_spectrum[:, np.newaxis]) / np.sum(power_spectrum))
        skewness = skew(power_spectrum)
        slope, _, _, _, _ = linregress(frequency_axis, log_power_spectrum)
        decrease = np.zeros(n_frames)
        for i in range(1, n_frames):
            decrease[i] = np.sum(energy[i:] - energy[i - 1]) / (n_frames - i)
            
        peak_amplitude = np.max(np.abs(y))
        diff = np.diff(y)  # Compute the differences between consecutive samples
        mean_diff = np.mean(np.abs(diff))  # Calculate the mean absolute difference
        smoothness = mean_diff / peak_amplitude 
        
        
        features = []
        features = file
        #features = (file[:-4] +'.' + str(i) + '.wav')
        features = np.append(features,len(y))
        features = np.append(features,np.mean(librosa.feature.chroma_stft(y=y,sr=sr)))
        features = np.append(features,np.var(librosa.feature.chroma_stft(y=y,sr=sr)))
        features = np.append(features, np.mean(librosa.feature.rms(y=y).flatten()))
        features = np.append(features, np.var(librosa.feature.rms(y=y).flatten()))
        features = np.append(features,librosa.feature.spectral_centroid(y=y,sr=sr).mean())
        features = np.append(features,librosa.feature.spectral_centroid(y=y,sr=sr).var())
        features = np.append(features,librosa.feature.spectral_bandwidth(y=y,sr=sr).mean())
        features = np.append(features,librosa.feature.spectral_bandwidth(y=y,sr=sr).var())
        features = np.append(features,librosa.feature.spectral_rolloff(y=y,sr=sr).mean())
        features = np.append(features,librosa.feature.zero_crossing_rate(y=y).mean())
        harmony, perceptual = librosa.effects.hpss(y=y)
        features = np.append(features,harmony.var())
        features = np.append(features,perceptual.mean()); features = np.append(features,perceptual.var())
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        features = np.append(features,np.mean(mfccs,axis=1).reshape(-1,1))
        features = np.append(features,np.var(mfccs,axis=1).reshape(-1,1))
        
        contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
        features = np.append(features,np.mean(contrast));
        features = np.append(features,np.var(contrast)) #spectral contrast mean + var
        stft = librosa.stft(y=y);  magnitude = np.abs(stft); flux = librosa.onset.onset_strength(S=magnitude,sr=sr); 
        features = np.append(features, np.mean(flux)); features = np.append(features,np.var(flux)) #spectral flux mean + var
        crest = np.max(magnitude, axis=0) / (np.sum(magnitude, axis=0) + 1e-10)
        features = np.append(features, np.mean(crest)); 
        flatness = librosa.feature.spectral_flatness(y=y)
        features = np.append(features, np.var(flatness)) #spectral flatness mean + var
        features = np.append(features, np.mean(decrease)); 
        features = np.append(features, slope); #spectral slope 
        features = np.append(features, skewness); #spectral skewness
        features = np.append(features, spread); #spectral spread 
        features = np.append(features, spectral_entropy); #spectral entropy 
        features = np.append(features, smoothness); # peak smoothness
        
        features= np.append(features,folder)
    
        return features

    def read_file(file_path):
        y, sr = librosa.load(file_path)

        return y,sr

    path = np.sort(os.listdir());
    names=[]; labels = []

    test = np.empty((1,2*n_mfcc + 28 )); j=0
     # iterate through all file
    for folder in path:
        # Check whether file is in text format or not
        os.chdir(general_path+'/'+folder)
        for file in np.sort(os.listdir()):
            if file.endswith(".wav") and file != 'jazz.00054.wav':
                file_path = f"{general_path}/{folder}/{file}"           
               
                y,sr = read_file(file_path)
                
                leny = len(y);
                count=0; total = 0; i =0;
                while (i < n_windows):
                    window = y[int(count):int(count+wsize)];
                    test = np.vstack((test,compute_features(window,sr,i,folder,file)))
                    i=i+1; count = count + overlap; total = count + wsize;
                    
                print(test.shape)
                j=j+1
                
                print("file number: {}".format(j))
    
    print('Extraction complete, now saving into a DataFrame. ')
    
      
    return test

In [59]:
test= features_overlap()

(38, 78)
file number: 1
(75, 78)
file number: 2


KeyboardInterrupt: 

In [22]:
columns = []
columns.extend(['filename','length'])
columns.extend(['chroma_stft_mean_' , 'chroma_stft_var_' ,'rms_mean_' ,'rms_var_' , 
                  'spectral_centroid_mean_' , 'spectral_centroid_var_' , 
              'spectral_bandwidth_mean_' , 'spectral_bandwidth_var_' ,'rolloff_mean_' ,'zero_crossing_rate_mean_' ,
              'harmony_var_' , 'perceptr_mean_' ,'perceptr_var_' ])
for j in range(25):
    columns.append('mfcc' + str(j+1) + '_mean_')
    columns.append('mfcc' + str(j+1) + '_var_')
            
columns.extend(['spectral_contrast_mean_', 'spectral_contrast_var_','spectral_flux_mean_',
                       'spectral_flux_var_','spectral_crest_mean_',
                    'spectral_flatness_var_','spectral_decrease_mean_', 'spectral_slope_',
                       'spectral_skewness_', 'spectral_spread_', 'spectral_entropy_','peak_smoothness_'])
        
columns.append('label')
    
features = pd.DataFrame(test, columns = columns)
features = features.drop(index=0)
features= features.reset_index()
features = features.drop(columns='index')
features.to_csv('/kaggle/working/features_overlap_3s_0.75s_2.csv')

78

In [2]:
df = pd.read_csv('/kaggle/input/overlap-tests/features_overlap_3s_0.75s.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36954 entries, 0 to 36953
Data columns (total 79 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                36954 non-null  int64  
 1   filename                  36954 non-null  object 
 2   length                    36954 non-null  int64  
 3   chroma_stft_mean_         36954 non-null  float64
 4   chroma_stft_var_          36954 non-null  float64
 5   rms_mean_                 36954 non-null  float64
 6   rms_var_                  36954 non-null  float64
 7   spectral_centroid_mean_   36954 non-null  float64
 8   spectral_centroid_var_    36954 non-null  float64
 9   spectral_bandwidth_mean_  36954 non-null  float64
 10  spectral_bandwidth_var_   36954 non-null  float64
 11  rolloff_mean_             36954 non-null  float64
 12  zero_crossing_rate_mean_  36954 non-null  float64
 13  harmony_var_              36954 non-null  float64
 14  percep

In [3]:
def grid_search(model, param_grid, X_train, y_train, X_val, y_val):
    val_index = [-1]*len(X_train) + [0]*len(X_val)
    gs = GridSearchCV(model, param_grid, refit=True, cv=PredefinedSplit(val_index), n_jobs = -1,scoring='accuracy')
    gs.fit(np.r_[X_train, X_val], np.r_[y_train, y_val])
    return gs

In [60]:
import warnings
warnings.filterwarnings('ignore')
import sklearn

def group_division(df):

    data = pd.read_csv('/kaggle/input/gtzan-dataset-music-genre-classification/Data/features_30_sec.csv')
    a = sklearn.utils.shuffle(data,random_state=0)
    a = a.drop(554)

    labels = np.unique(a['label'])
    files = []
    for i in labels:
        for k in range(80):
            files.append((a[a['label']== i].iloc[k]['filename']))
   
    test_30df = a.copy()
    
    for i in files: 
        test_30df = test_30df.drop(test_30df[test_30df['filename'] == i].index) ### salva os 199 arquivos de teste

    train_val30 = a.drop(test_30df.index)
        
    y_30 = test_30df['label']

    for i in range(800):
        files[i] = (files[i])[:-4] + '.1024.wav'
    
    train_val = pd.DataFrame()
    for i in files:
        train_val = pd.concat([train_val,df.loc[df['filename'] == i]])
    test= df.drop(train_val.index)    

    b = sklearn.utils.shuffle(train_val30,random_state=0)
    
    files = []
    for i in labels:
        for k in range(60):
            files.append((b[b['label']== i].iloc[k]['filename']))

    for i in range(600):
        files[i] = (files[i])[:-4] + '.1024.wav'

        
    train_val = train_val.drop(columns='Unnamed: 0')
    test = test.drop(columns='Unnamed: 0')

    train = pd.DataFrame()
    for i in files:
        train = pd.concat([train,train_val.loc[train_val['filename'] == i]])
    val = train_val.drop(train.index)

    X_tv = train_val.drop(columns=['label','filename','length']); y_tv = train_val['label']
    X_train = train.drop(columns=['label','filename','length']); y_train = train['label']
    X_val = val.drop(columns=['label','filename','length']); y_val = val['label']
    X_test = test.drop(columns=['label','filename','length']); y_test = test['label']
    scaler = StandardScaler(); scaler.fit(X_tv); X_tv_scaled = scaler.transform(X_tv); X_test_scaled = scaler.transform(X_test);
    scaler.fit(X_train); 
    X_train_scaled = scaler.transform(X_train); X_val_scaled = scaler.transform(X_val); 
    
    #return train_val,test
    return X_tv, X_tv_scaled, X_test, X_test_scaled, X_train_scaled, X_val_scaled, y_tv, y_test, y_train, y_val, train_val,test, y_30

#### Teste 1: janelas de 3s e overlap de 0.75s

In [104]:
df = pd.read_csv('/kaggle/input/overlap-tests/features_overlap_3s_0.75s.csv')
X_tv, X_tv_scaled, X_test, X_test_scaled, X_train_scaled, X_val_scaled, y_tv, y_test, y_train, y_val, train_val,test, y_30 = group_division(df)
#tv, train, val = group_division(df)

In [8]:
param_grid = {'C':     10.0**np.arange(-3, 4.01, 1), 
              'gamma': 10.0**np.arange(-5, 0.01, 1)}
model = SVC(kernel='rbf', max_iter = 20000,random_state=0);
gs_svm = grid_search(model, param_grid, X_train_scaled, y_train, X_val_scaled, y_val)
print('best_score:', gs_svm.best_score_)
print('best_params:', gs_svm.best_params_)

/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=20000).  Con

best_score: 0.7176645492634139
best_params: {'C': 10.0, 'gamma': 0.01}


In [108]:
model = SVC(random_state=0,max_iter=20000, **gs_svm.best_params_, probability=True);
model.fit(X_train_scaled, y_train);

print('Optimized: SVM')
print(f'Val accuracy: {accuracy_score(y_val, model.predict(X_val_scaled)):.1%}')
print(f'Test accuracy: {accuracy_score(y_test, model.predict(X_test_scaled)):.1%}')
model.fit(X_tv_scaled, y_tv);
print(f'Test refited accuracy: {accuracy_score(y_test, model.predict(X_test_scaled)):.1%}')

Optimized: SVM
Val accuracy: 71.8%
Test accuracy: 75.9%
Test refited accuracy: 77.3%


In [12]:
model = MLPClassifier(hidden_layer_sizes=(1000,500,200), max_iter = 2000,activation = 'relu', solver = 'adam',learning_rate = 'adaptive',
                      random_state=0, alpha = 1e-5)
model.fit(X_train_scaled,y_train)

print(f'Validation accuracy: {accuracy_score(y_val, model.predict(X_val_scaled)):.1%}')
print(f'Test accuracy: {accuracy_score(y_test, model.predict(X_test_scaled)):.1%}')
model.fit(X_tv_scaled,y_tv)
print(f'Test accuracy: {accuracy_score(y_test, model.predict(X_test_scaled)):.1%}')

Validation accuracy: 72.6%
Test accuracy: 75.4%
Test accuracy: 76.2%


In [64]:
def ensemble_prob():
    
    lens = []
    fnames = np.unique((test['filename']))
    for i in fnames:
        lens.append(len(test[test['filename'] == i]))
    lens = np.array(lens)
    y_probs_test = model.predict_proba(X_test_scaled)
    i = 0; y_probs = np.empty((1,10))
    
    for j in lens:
        y_probs = np.vstack((y_probs,sum(y_probs_test[i:(j+i)])/j))
        i = i+j
    y_probs = y_probs[1:]
    
    y_pred=[]
    
    for i in y_probs:
        y_pred.append(np.argmax(i))
    
    y_test_ord = np.sort(y_30)

    from sklearn.preprocessing import OrdinalEncoder
    enc = OrdinalEncoder()
    y_test_ord = enc.fit_transform(np.array(y_test_ord).reshape(-1,1));

    print(f'Ensemble Test accuracy: {accuracy_score(y_test_ord, y_pred):.1%}')
    
    return y_pred, y_test_ord

In [2]:
def ensemble_prob_corr(M=37):
    y_probs_test = model.predict_proba(X_test_scaled)
    y_probs_test = np.reshape(y_probs_test,(199,M,10))
    y_probs = np.mean(y_probs_test,axis=1)    
    y_pred=[]
    
    for i in y_probs:
        y_pred.append(np.argmax(i))
    y_test_ord = np.sort(y_30)

    from sklearn.preprocessing import OrdinalEncoder
    enc = OrdinalEncoder()
    y_test_ord = enc.fit_transform(np.array(y_test_ord).reshape(-1,1));
    print(f'Ensemble Test accuracy: {accuracy_score(y_test_ord, y_pred):.1%}')
    
    return y_pred, y_test_ord

In [114]:
y_pred, y_test_ord = ensemble_prob()

Ensemble Test accuracy: 86.9%


#### Teste 2:  janela 5s - 0.75s overlap: 

In [91]:
df = pd.read_csv('/kaggle/input/overlap-tests/features_overlap_5s_0.75s.csv')
X_tv, X_tv_scaled, X_test, X_test_scaled, X_train_scaled, X_val_scaled, y_tv, y_test, y_train, y_val, train_val,test, y_30 = group_division(df)

In [100]:
model = SVC(random_state=0,max_iter=100000,C=10,gamma=0.01,probability=True);
#model.fit(X_train_scaled, y_train);

#print('Optimized: SVM')
#print(f'Val accuracy: {accuracy_score(y_val, model.predict(X_val_scaled)):.1%}')
#print(f'Test accuracy: {accuracy_score(y_test, model.predict(X_test_scaled)):.1%}')
model.fit(X_tv_scaled, y_tv);
print(f'Test refited accuracy: {accuracy_score(y_test, model.predict(X_test_scaled)):.1%}')

Test refited accuracy: 78.7%


In [103]:
y_pred, y_test_ord = ensemble_prob()

Ensemble Test accuracy: 86.4%


#### Teste 3: verificando se está sendo extraído da forma correta:

In [42]:
df = pd.read_csv('/kaggle/input/overlap-tests/features_overlap_3s_0.75s_2.csv')

In [43]:
df

,Unnamed: 0,filename,length,chroma_stft_mean_,chroma_stft_var_,rms_mean_,rms_var_,spectral_centroid_mean_,spectral_centroid_var_,spectral_bandwidth_mean_,...,spectral_flux_var_,spectral_crest_mean_,spectral_flatness_var_,spectral_decrease_mean_,spectral_slope_,spectral_skewness_,spectral_spread_,spectral_entropy_,peak_smoothness_,label
0,0,blues.00000.wav,66150,0.335555,0.090997,0.130189,0.003559,1773.358004,169450.829520,1972.334258,...,0.052980,0.041710,0.000106,-0.000065,-0.000576,6.658046,4936.666502,6.750400,0.032151,blues
1,1,blues.00000.wav,66150,0.338329,0.092477,0.128709,0.002610,1833.892636,191813.557216,2018.016509,...,0.043479,0.041203,0.000054,-0.000060,-0.000558,7.073728,4920.012309,6.749108,0.037376,blues
2,2,blues.00000.wav,66150,0.340994,0.089081,0.114578,0.001646,1809.237614,207162.698818,1940.803912,...,0.037833,0.038637,0.000054,-0.000045,-0.000560,6.692923,4907.074443,6.868812,0.038511,blues
3,3,blues.00000.wav,66150,0.333729,0.088401,0.118732,0.002061,1817.257615,191135.085276,1992.283022,...,0.048178,0.039604,0.000050,-0.000051,-0.000557,6.271461,4947.549468,6.821972,0.037020,blues
4,4,blues.00000.wav,66150,0.343533,0.086789,0.112113,0.001492,1816.983867,92075.628328,2009.914524,...,0.046457,0.039681,0.000039,-0.000044,-0.000565,8.310191,4948.643588,6.807990,0.036430,blues
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36958,36958,rock.00099.wav,66150,0.386742,0.084580,0.065981,0.000313,2087.720818,201428.676122,2022.936048,...,0.006440,0.039489,0.000026,-0.000014,-0.000595,23.281252,4732.019659,6.224523,0.047985,rock
36959,36959,rock.00099.wav,66150,0.371495,0.087118,0.054499,0.000136,1725.026553,245449.116927,1889.229424,...,0.002791,0.047645,0.000016,-0.000010,-0.000595,20.632673,4955.978396,5.750731,0.032841,rock
36960,36960,rock.00099.wav,66150,0.350483,0.089776,0.047872,0.000039,1482.510129,167700.119169,1842.390196,...,0.000579,0.059669,0.000008,-0.000007,-0.000574,14.700179,5151.335140,5.276535,0.029511,rock
36961,36961,rock.00099.wav,66150,0.365628,0.090727,0.048403,0.000047,1501.029504,351342.579782,1821.558636,...,0.002519,0.059824,0.000051,-0.000007,-0.000525,10.382643,5126.534624,5.672218,0.026605,rock


In [44]:
36963/37

999.0

In [3]:
import warnings
warnings.filterwarnings('ignore')

import sklearn

def group_division_corr(df):

    data = pd.read_csv('/kaggle/input/gtzan-dataset-music-genre-classification/Data/features_30_sec.csv')
    a = sklearn.utils.shuffle(data,random_state=0)
    a = a.drop(554)

    labels = np.unique(a['label'])
    files = []
    for i in labels:
        for k in range(80):
            files.append((a[a['label']== i].iloc[k]['filename']))
   
    test_30df = a.copy()
    
    for i in files: 
        test_30df = test_30df.drop(test_30df[test_30df['filename'] == i].index) ### salva os 199 arquivos de teste

    train_val30 = a.drop(test_30df.index); y_30 = test_30df['label']

    train_val = pd.DataFrame()
    for i in files:
        train_val = pd.concat([train_val,df.loc[df['filename'] == i]])
        
    test= df.drop(train_val.index)
    
    b = sklearn.utils.shuffle(train_val30,random_state=0)
    
    files = []
    for i in labels:
        for k in range(60):
            files.append((b[b['label']== i].iloc[k]['filename']))

    train_val = train_val.drop(columns='Unnamed: 0')
    test = test.drop(columns='Unnamed: 0')

    train = pd.DataFrame()
    for i in files:
        train = pd.concat([train,train_val.loc[train_val['filename'] == i]])
    val = train_val.drop(train.index)
    X_tv = train_val.drop(columns=['label','filename','length']); y_tv = train_val['label']
    X_train = train.drop(columns=['label','filename','length']); y_train = train['label']
    X_val = val.drop(columns=['label','filename','length']); y_val = val['label']
    X_test = test.drop(columns=['label','filename','length']); y_test = test['label']
    scaler = StandardScaler(); scaler.fit(X_tv); X_tv_scaled = scaler.transform(X_tv); X_test_scaled = scaler.transform(X_test);
    scaler.fit(X_train); 
    X_train_scaled = scaler.transform(X_train); X_val_scaled = scaler.transform(X_val); 
    
    return X_tv, X_tv_scaled, X_test, X_test_scaled, X_train_scaled, X_val_scaled, y_tv, y_test, y_train, y_val, train_val,test, y_30

In [46]:
X_tv, X_tv_scaled, X_test, X_test_scaled, X_train_scaled, X_val_scaled, y_tv, y_test, y_train, y_val, train_val,test, y_30 = group_division_corr(df)

In [29]:
model = SVC(random_state=0,max_iter=20000, C=10, gamma=0.01, probability=True);
model.fit(X_train_scaled, y_train);

print('Optimized: SVM')
print(f'Val accuracy: {accuracy_score(y_val, model.predict(X_val_scaled)):.1%}')
print(f'Test accuracy: {accuracy_score(y_test, model.predict(X_test_scaled)):.1%}')
model.fit(X_tv_scaled, y_tv);
print(f'Test refited accuracy: {accuracy_score(y_test, model.predict(X_test_scaled)):.1%}')

Optimized: SVM
Val accuracy: 71.8%
Test accuracy: 75.8%
Test refited accuracy: 77.4%


In [47]:
y_pred, y_test_ord = ensemble_prob_corr(37)

Ensemble Test accuracy: 86.4%


#### Teste 4: janelas de 10s - overlap 1s

In [54]:
df = pd.read_csv('/kaggle/input/overlap-tests/features_overlap_10s_1s.csv')

In [55]:
df

,Unnamed: 0,filename,length,chroma_stft_mean_,chroma_stft_var_,rms_mean_,rms_var_,spectral_centroid_mean_,spectral_centroid_var_,spectral_bandwidth_mean_,...,spectral_flux_var_,spectral_crest_mean_,spectral_flatness_var_,spectral_decrease_mean_,spectral_slope_,spectral_skewness_,spectral_spread_,spectral_entropy_,peak_smoothness_,label
0,0,blues.00000.1024.wav,220500,0.342070,0.089073,0.124413,0.003156,1782.836092,116521.944647,2020.991068,...,0.049386,0.045294,0.000024,-0.000195,-0.000567,10.298336,4966.679165,6.666932,0.030116,blues
1,1,blues.00000.1024.wav,220500,0.336136,0.089443,0.127287,0.002864,1778.517754,124837.271644,2020.221924,...,0.047621,0.045851,0.000024,-0.000199,-0.000567,9.603058,4965.198436,6.700722,0.032264,blues
2,2,blues.00000.1024.wav,220500,0.345832,0.088920,0.125595,0.002884,1763.843249,134933.869862,2012.385271,...,0.049840,0.045407,0.000024,-0.000194,-0.000567,8.652051,4975.401306,6.702133,0.031419,blues
3,3,blues.00000.1024.wav,220500,0.349359,0.088285,0.127411,0.002809,1753.206046,103317.632181,2023.283867,...,0.050269,0.045344,0.000018,-0.000199,-0.000571,9.091313,4978.241607,6.666284,0.031359,blues
4,4,blues.00000.1024.wav,220500,0.346886,0.088866,0.129217,0.002750,1724.243747,97195.852212,2022.121447,...,0.050970,0.046724,0.000011,-0.000204,-0.000569,8.319227,4996.210985,6.608401,0.030684,blues
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20964,20964,rock.00099.1024.wav,220500,0.368592,0.083650,0.057992,0.000390,1698.094907,468521.773727,1785.807450,...,0.008313,0.055611,0.000017,-0.000038,-0.000575,13.958147,4789.588479,6.580087,0.034257,rock
20965,20965,rock.00099.1024.wav,220500,0.366748,0.084541,0.058350,0.000381,1735.133509,438001.814095,1820.047215,...,0.008203,0.055114,0.000016,-0.000038,-0.000576,15.447082,4791.357200,6.543554,0.034506,rock
20966,20966,rock.00099.1024.wav,220500,0.369905,0.085521,0.057467,0.000379,1694.625572,471368.613528,1804.509425,...,0.008139,0.056909,0.000016,-0.000037,-0.000574,16.606852,4797.857225,6.467248,0.033207,rock
20967,20967,rock.00099.1024.wav,220500,0.367195,0.085734,0.055805,0.000374,1677.682900,465329.512073,1803.301077,...,0.008415,0.058451,0.000026,-0.000035,-0.000580,17.740279,4796.831631,6.421790,0.031988,rock


In [61]:
X_tv, X_tv_scaled, X_test, X_test_scaled, X_train_scaled, X_val_scaled, y_tv, y_test, y_train, y_val, train_val,test, y_30 = group_division(df)
#tv,test = group_division(df)

In [62]:
model = SVC(random_state=0,max_iter=20000, C=10, gamma=0.01, probability=True);
model.fit(X_train_scaled, y_train);

print('Optimized: SVM')
print(f'Val accuracy: {accuracy_score(y_val, model.predict(X_val_scaled)):.1%}')
print(f'Test accuracy: {accuracy_score(y_test, model.predict(X_test_scaled)):.1%}')
model.fit(X_tv_scaled, y_tv);
print(f'Test refited accuracy: {accuracy_score(y_test, model.predict(X_test_scaled)):.1%}')

Optimized: SVM
Val accuracy: 73.2%
Test accuracy: 76.0%
Test refited accuracy: 77.8%


In [65]:
y_pred, y_test_ord = ensemble_prob()

Ensemble Test accuracy: 82.4%


#### Teste 5: janelas de 3s - overlap de 1.5s

In [68]:
df = pd.read_csv('/kaggle/input/overlap-tests/features_overlap_3s_1.5s.csv')
X_tv, X_tv_scaled, X_test, X_test_scaled, X_train_scaled, X_val_scaled, y_tv, y_test, y_train, y_val, train_val,test, y_30 = group_division_corr(df)

In [74]:
model = SVC(random_state=0,max_iter=20000, C=10, gamma=0.01, probability=True);
model.fit(X_train_scaled, y_train);

print('Optimized: SVM')
print(f'Val accuracy: {accuracy_score(y_val, model.predict(X_val_scaled)):.1%}')
print(f'Test accuracy: {accuracy_score(y_test, model.predict(X_test_scaled)):.1%}')
model.fit(X_tv_scaled, y_tv);
print(f'Test refited accuracy: {accuracy_score(y_test, model.predict(X_test_scaled)):.1%}')

Optimized: SVM
Val accuracy: 70.9%
Test accuracy: 75.3%
Test refited accuracy: 77.2%


In [75]:
y_pred, y_test_ord = ensemble_prob_corr(M=19)

Ensemble Test accuracy: 86.4%


#### Teste 6: janelas  de 3s - overlap 0.375s

In [5]:
df = pd.read_csv('/kaggle/input/overlap-tests/features_overlap_3s_0.375s.csv')
X_tv, X_tv_scaled, X_test, X_test_scaled, X_train_scaled, X_val_scaled, y_tv, y_test, y_train, y_val, train_val,test, y_30 = group_division_corr(df)

In [6]:
model = SVC(random_state=0,max_iter=20000, C=10, gamma=0.01, probability=True);
model.fit(X_train_scaled, y_train);

print('Optimized: SVM')
print(f'Val accuracy: {accuracy_score(y_val, model.predict(X_val_scaled)):.1%}')
print(f'Test accuracy: {accuracy_score(y_test, model.predict(X_test_scaled)):.1%}')
model.fit(X_tv_scaled, y_tv);
print(f'Test refited accuracy: {accuracy_score(y_test, model.predict(X_test_scaled)):.1%}')

Optimized: SVM
Val accuracy: 72.0%
Test accuracy: 75.7%
Test refited accuracy: 77.7%


In [7]:
y_pred, y_test_ord = ensemble_prob_corr(M=73)

Ensemble Test accuracy: 87.9%


#### Teste 7: janelas de 10s - overlap 0.5s:

In [4]:
df = pd.read_csv('/kaggle/input/overlap-tests/features_overlap_10s_0.5s.csv')
X_tv, X_tv_scaled, X_test, X_test_scaled, X_train_scaled, X_val_scaled, y_tv, y_test, y_train, y_val, train_val,test, y_30 = group_division_corr(df)

In [5]:
model = SVC(random_state=0,max_iter=20000, C=10, gamma=0.01, probability=True);
model.fit(X_train_scaled, y_train);

print('Optimized: SVM')
print(f'Val accuracy: {accuracy_score(y_val, model.predict(X_val_scaled)):.1%}')
print(f'Test accuracy: {accuracy_score(y_test, model.predict(X_test_scaled)):.1%}')
model.fit(X_tv_scaled, y_tv);
print(f'Test refited accuracy: {accuracy_score(y_test, model.predict(X_test_scaled)):.1%}')

Optimized: SVM
Val accuracy: 73.0%
Test accuracy: 76.4%
Test refited accuracy: 77.8%


In [7]:
y_pred, y_test_ord = ensemble_prob_corr(M=41)

Ensemble Test accuracy: 81.4%


#### Teste 8: janelas 15s - overlap 0.5s

In [8]:
df = pd.read_csv('/kaggle/input/overlap-tests/features_overlap_15s_0.5s.csv')
X_tv, X_tv_scaled, X_test, X_test_scaled, X_train_scaled, X_val_scaled, y_tv, y_test, y_train, y_val, train_val,test, y_30 = group_division_corr(df)

In [9]:
model = SVC(random_state=0,max_iter=20000, C=10, gamma=0.01, probability=True);
model.fit(X_train_scaled, y_train);

print('Optimized: SVM')
print(f'Val accuracy: {accuracy_score(y_val, model.predict(X_val_scaled)):.1%}')
print(f'Test accuracy: {accuracy_score(y_test, model.predict(X_test_scaled)):.1%}')
model.fit(X_tv_scaled, y_tv);
print(f'Test refited accuracy: {accuracy_score(y_test, model.predict(X_test_scaled)):.1%}')

Optimized: SVM
Val accuracy: 73.0%
Test accuracy: 76.7%
Test refited accuracy: 76.5%


In [10]:
y_pred, y_test_ord = ensemble_prob_corr(M=31)

Ensemble Test accuracy: 76.9%


#### Teste 9: janelas 20s - overlap 0.5

In [11]:
df = pd.read_csv('/kaggle/input/overlap-tests/features_overlap_20s_0.5s.csv')
X_tv, X_tv_scaled, X_test, X_test_scaled, X_train_scaled, X_val_scaled, y_tv, y_test, y_train, y_val, train_val,test, y_30 = group_division_corr(df)

In [12]:
model = SVC(random_state=0,max_iter=20000, C=10, gamma=0.01, probability=True);
model.fit(X_train_scaled, y_train);

print('Optimized: SVM')
print(f'Val accuracy: {accuracy_score(y_val, model.predict(X_val_scaled)):.1%}')
print(f'Test accuracy: {accuracy_score(y_test, model.predict(X_test_scaled)):.1%}')
model.fit(X_tv_scaled, y_tv);
print(f'Test refited accuracy: {accuracy_score(y_test, model.predict(X_test_scaled)):.1%}')

Optimized: SVM
Val accuracy: 72.8%
Test accuracy: 75.7%
Test refited accuracy: 76.0%


In [13]:
y_pred, y_test_ord = ensemble_prob_corr(M=21)

Ensemble Test accuracy: 76.4%
